In [102]:
import tensorflow as tf
import numpy as np
import pickle as pkl
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [103]:
with open('Word2id', 'rb') as f:
    word2id = pkl.load(f)
    f.close()

with open('Labels', 'rb') as f:
    label = pkl.load(f)
    f.close()

with open('Fixed_Lenth_Data', 'rb') as f:
    data = pkl.load(f)
    f.close()

In [4]:
with open('Embeddings', 'rb') as f:
    emb = pkl.load(f)
    f.close()

In [104]:
emb = np.asarray(emb)

In [105]:
emb_mat = np.zeros((len(word2id), 50))

In [106]:
emb_mat.shape

(400543, 50)

In [107]:
emb.shape

(400543, 50)

In [108]:
len(word2id)

400543

In [109]:
ndim = 50
sen_len = 10
emb_layer = keras.layers.Embedding(400543, ndim, weights=[emb], input_length=sen_len, trainable=True)

In [110]:
emb_layer

In [111]:
type(data[0][0])

int

In [112]:
input_sentence = keras.Input(shape=(sen_len,), dtype = 'int32')

In [113]:
input_sentence

<tf.Tensor 'input_8:0' shape=(None, 10) dtype=int32>

In [114]:
input_sequence = emb_layer(input_sentence)

In [115]:
input_sequence

<tf.Tensor 'embedding_5/Identity:0' shape=(None, 10, 50) dtype=float32>

In [116]:
lstm1 = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True))(input_sequence)

In [117]:
lstm1

<tf.Tensor 'bidirectional_12/Identity:0' shape=(None, 10, 100) dtype=float32>

In [118]:
lstm2 = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True))([lstm1])

In [119]:
sentence_vector = keras.layers.Flatten()(lstm2)

In [120]:
sentence_vector

<tf.Tensor 'flatten_3/Identity:0' shape=(None, 1000) dtype=float32>

In [121]:
fc1 = keras.layers.Dense(500)(sentence_vector)

In [122]:
fc1

<tf.Tensor 'dense_7/Identity:0' shape=(None, 500) dtype=float32>

In [123]:
fc2 = keras.layers.Dense(100)(fc1)

In [124]:
fc2

<tf.Tensor 'dense_8/Identity:0' shape=(None, 100) dtype=float32>

In [125]:
output = keras.layers.Dense(4, activation='softmax')(fc2)

In [126]:
output

<tf.Tensor 'dense_9/Identity:0' shape=(None, 4) dtype=float32>

In [127]:
model = keras.Model(input_sentence, output)

In [128]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [133]:
x_train, x_test, y_train, y_test = train_test_split(data, label, shuffle=True, stratify=label, random_state = 0, test_size = 0.2)

In [134]:
len(x_train)

6080

In [135]:
len(y_train)

6080

In [138]:
for i in range(10):
    x_temp, x_val, y_temp, y_val = train_test_split(x_train, y_train, shuffle=True, stratify=y_train, test_size = 0.2)
    model.fit(x_temp, y_temp, epochs=1, batch_size=50)
    model.evaluate(x_val, y_val)

Train on 4864 samples
1216/1216 [==============================] - 1s 1ms/sample - loss: 0.5039 - acc: 0.8281
Train on 4864 samples
1216/1216 [==============================] - 0s 183us/sample - loss: 0.4084 - acc: 0.8602
Train on 4864 samples
1216/1216 [==============================] - 0s 184us/sample - loss: 0.3099 - acc: 0.8956
Train on 4864 samples
1216/1216 [==============================] - 0s 182us/sample - loss: 0.2935 - acc: 0.9005
Train on 4864 samples
1216/1216 [==============================] - 0s 183us/sample - loss: 0.1902 - acc: 0.9383
Train on 4864 samples
1216/1216 [==============================] - 0s 184us/sample - loss: 0.2034 - acc: 0.9186
Train on 4864 samples
1216/1216 [==============================] - 0s 180us/sample - loss: 0.0849 - acc: 0.9704
Train on 4864 samples
1216/1216 [==============================] - 0s 180us/sample - loss: 0.1137 - acc: 0.9638
Train on 4864 samples
1216/1216 [==============================] - 0s 183us/sample - loss: 0.0514 - acc: 0

In [139]:
model.evaluate(x_test, y_test)

1520/1520 [==============================] - 1s 860us/sample - loss: 0.9188 - acc: 0.8289


[0.9188498716605337, 0.82894737]

In [141]:
keras.models.save_model(model, "noob_lstm.h5")

In [145]:
model2 = keras.models.load_model("noob_lstm.h5", compile=False)

In [144]:
model2.evaluate(x_test, y_test)

1520/1520 [==============================] - 1s 507us/sample - loss: 0.9188 - acc: 0.8289


[0.9188498716605337, 0.82894737]